In [ ]:
import pandas as pd
import numpy as np
import requests
import json
import os
# Gets every bill whether it passed or not

propublica_token = os.environ['propub_token']

memberurl = 'https://voteview.com/static/data/out/members/HS117_members.csv'
cvoteurl = 'https://voteview.com/static/data/out/rollcalls/HS117_rollcalls.csv'
memvoteurl = 'https://voteview.com/static/data/out/votes/HS117_votes.csv'
cpartyurl = 'https://voteview.com/static/data/out/parties/HS117_parties.csv'

members_vv = pd.read_csv(memberurl)
cvote_vv = pd.read_csv(cvoteurl)
memvote_vv = pd.read_csv(memvoteurl)
cparty_vv = pd.read_csv(cpartyurl)

useragent_url = 'https://httpbin.org/user-agent'
r = requests.get(useragent_url)
useragent = json.loads(r.text)['user-agent']

headers = {'X-API-Key': propublica_token,
          'User-Agent': useragent,
          'From': 'nqj5zk@virginia.edu'}

root = 'https://api.propublica.org'
congress = '117'
chamber = 'house'
memberendpoint = '/congress/v1/{congress}/{chamber}/members.json'.format(congress=congress, chamber=chamber)

r = requests.get(root + memberendpoint, headers = headers)
myjson = json.loads(r.text)
house_pp = pd.json_normalize(myjson, record_path = ['results', 'members'])

chamber = 'senate'
memberendpoint = '/congress/v1/{congress}/{chamber}/members.json'.format(congress=congress, chamber=chamber)
r = requests.get(root + memberendpoint, headers = headers)
myjson = json.loads(r.text)
senate_pp = pd.json_normalize(myjson, record_path = ['results', 'members'])


billsendpoint = '/congress/v1/{congress}/{chamber}/bills/{billtype}.json'.format(congress='117',
                                                                             chamber='both',
                                                                             billtype='introduced')
n = 20
i = 0
bills_117 = pd.DataFrame()
while n == 20:
    myparams = {'query': '', 'offset':i}
    r = requests.get(root + billsendpoint, headers = headers, params = myparams)
    myjson = json.loads(r.text)
    bills_pp = pd.json_normalize(myjson, record_path = ['results', 'bills'])
    bills_117 = pd.concat([bills_117, bills_pp], ignore_index=True)
    i = i + 20
    print(i)
    n = myjson['results'][0]['num_results']

In [ ]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 600)

In [ ]:
members_vv.sort_values('nominate_dim1', ascending=True)

In [ ]:
members_vv.query("chamber == 'Senate' & born<= 1950 ")

# bills_117 will list all bills passed or not 
bills_117['sponsor_name'].value_counts()
bills_117.groupby('sponsor_name').size().sort_values(ascending=True)

bills_117['house_passage'][0] is not None

bills_117['pass_house']=[x is not None for x in bills_117['house_passage']]
bills_117['pass_senate']=[x is not None for x in bills_117['senate_passage']]
bills_117['pass_enact']=[x is not None for x in bills_117['enacted']]

bills_117.groupby('sponsor_party').agg({'pass_house':'mean',
                                       'pass_senate' : 'mean',
                                        'pass_enact': 'mean'})

In [ ]:
members_vv.head(3)

In [ ]:
house_pp.head(3)

In [ ]:
senate_pp.head(3)

In [ ]:
# combining different dataframes 
members_pp = pd.concat([house_pp, senate_pp],ignore_index=True)
members_pp.head(3)

In [ ]:
members_pp.query("icpsr_id == '20301'")

In [ ]:
members_pp['icpsr']=members_pp['icpsr_id'].astype('float')

In [ ]:
# 1. locate the dataframes to merge
# 2. add the columns you want to join on 
# 3. 
members_total = pd.merge(members_vv, members_pp,
                        on = 'icpsr', 
                        how = 'outer',
                        #validate one-to-one should match the same amount of variables per other data frame
                        #validate = 'one_to_one',
                        indicator = 'matched')

In [ ]:
pd.set_option('display.max_columns', 80)
members_total.query("matched == 'right_only'")[members_pp.columns]

In [ ]:
members_pp['icpsr_id']

In [ ]:
members_pp.columns

In [ ]:
members_vv.columns

In [ ]:
members_vv[['bioname', 'born', 'state_abbrev']].head(3)

In [ ]:
members_pp[['last_name', 'date_of_birth', 'state']].head(3)

In [ ]:
members_vv['last_name']=members_vv['bioname'].str.split(pat=',', expand=True)[0]

In [ ]:
members_pp['last_name']=members_pp['last_name'].str.upper()

In [ ]:
members_pp['born']=members_pp['date_of_birth'].str.split(pat='-', expand=True)[0]

In [ ]:
# This kinda messes up because there are 3 people who match in name and birth
members_vv[['last_name', 'born', 'state_abbrev']].duplicated().value_counts()

In [ ]:
members_pp[['state', 'born','last_name']].duplicated().value_counts()

In [ ]:
members_vv['born']= members_vv['born'].astype('str')
members_pp['born']= members_pp['born'].astype('str')

In [ ]:
members_total = pd.merge(members_vv, member_pp,
                        left_on = ['last_name', 'born', 'state_abbrev'],
                        right_on = ['last_name','born','state'],
                        how = 'outer',
                        validate = 'one-to-one',
                        indicator = 'matched')          
                                   

In [ ]:
members_total['matched'].value_counts()

In [ ]:

members_total.query("matched == 'left_only'")['last_name', 'born', 'state_abbrev']